In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
# this directory is where I'm leaving files temporarily from the test server
csvDir = "Z:\Jack's stuff\FBTestServer"

In [ ]:
# probably should change these file names ...
# some active boms is the export of all active boms from FB
# FB_Aweto is the list of boms with labor data
# import bom sheet is the sheet saved at the end to be imported back into FB
csvFilepath = os.path.join(csvDir, 'some active boms.csv')
laborSheet = os.path.join(csvDir, 'FB_Aweto.csv')
saveSheet = os.path.join(csvDir, 'import bom sheet.csv')

In [ ]:
#csvFilepath = os.path.join(csvDir, 'BOMs.csv')

In [ ]:
# MFG labor types from the data module query used in FB_Auto_Scheduler
data = pd.read_csv(laborSheet)

In [ ]:
# BOM export from FB
# note that if this gives a warning about mixed datatype, the import will probably fail.
boms = pd.read_csv(csvFilepath)

In [ ]:
# column headers from the BOM export
flag = 'Flag'
bomNum = 'Number'
center = 'CF-Production Center'
setup = 'CF-Kitting Set-Up Time'
kit = 'CF-Kitting unit multiple'
assy = 'CF-Unit Assembly multiple'
test = 'CF-Test and QC Multiple'
pack = 'CF-Packaging/ Storage Prep. Multiple'
mach1 = 'CF-Machine 1'
mach1Setup = 'CF-Machine 1 Set up Time'
mach1Mult = 'CF-Machine 1 Run Time (Per Unit)'
mach2 = 'CF-Machine 2'
mach2Setup = 'CF-Machine 2 Set up Time'
mach2Mult = 'CF-Machine 2 Run Time (Per Unit)'

In [ ]:
# making a list of BOM columns for editing values
bomHeaderList = [center,
                 setup,
                 kit,
                 assy,
                 test,
                 pack,
                 mach1,
                 mach1Setup,
                 mach1Mult,
                 mach2,
                 mach2Setup,
                 mach2Mult]

In [ ]:
# making a list of columns use for reference when editing BOMs
dataHeaderList = ['Production Center',
                 'Kit Setup',
                 'Kit Mult',
                 'Assy Mult',
                 'Test Mult',
                 'Pack Mult',
                 'Machine 1',
                 'Machine 1 Setup',
                 'Machine 1 Mult',
                 'Machine 2',
                 'Machine 2 Setup',
                 'Machine 2 Mult']

In [ ]:
# have to cast these columns as strings or it thinks they're floats and fails to set a value
boms[[center, mach1, mach2]] = boms[[center, mach1, mach2]].copy().astype(str)

In [ ]:
# this loop looks through the BOM export for BOM lines, finds the comparable line in the data sheet,
#   and edits the BOM info accordingly.
# this is working but it will fail when it finds a BOM from the export that's missing from the labor estimate list
for line in range(0, len(boms)):
    if boms[flag].iat[line] == 'BOM':
        num = boms[bomNum].iat[line]
        info = data[data['BOM'] == num].copy()
        if len(info) > 0:
            for header in range(0, len(bomHeaderList)):
                print(line)
                print(header)
                boms[bomHeaderList[header]].iat[line] = info[dataHeaderList[header]].iat[0]

In [ ]:
# pandas filled NaN values with 'nan' strings when these columns were cast as strings.
# this just reverts what's left.
boms[center].replace(to_replace='nan', value=np.nan, inplace=True)
boms[mach1].replace(to_replace='nan', value=np.nan, inplace=True)
boms[mach2].replace(to_replace='nan', value=np.nan, inplace=True)

In [ ]:
# trying to remove all the extra BOMs from the list to speed up import and avoid data issues
boms.reset_index(drop=True, inplace=True)

In [ ]:
boms['Destiny'] = 'keep'

In [ ]:
destinySetter = 'keep'

In [ ]:
for line in range (0, len(boms)):
    if boms[flag].iat[line] == 'BOM':
        num = boms[bomNum].iat[line]
        info = data[data['BOM'] == num].copy()
        if len(info) > 0:
            destinySetter = 'keep'
        else:
            destinySetter = 'drop'
    boms['Destiny'].iat[line] = destinySetter

In [ ]:
boms = boms[boms['Destiny'] == 'keep'].copy()

In [ ]:
boms.drop('Destiny', axis=1, inplace=True)

In [ ]:
# this worked ... I'm just surprised.
boms.to_csv(saveSheet, index=False)


In [ ]:
# This is an example of the SQL being used to get reference data.  This needs to be hand updated with labor
# estimates before finishing this script
"""
SELECT PART.NUM as "Part",
        BOM.NUM as "BOM",
        CSMAKEBUY.INFO as "MAKEBUY",
        CSPRODCENTER.INFO as "Production Center",
        CSKITSETUP.INFO as "Kit Setup",
        CSKITMULT.INFO as "Kit Mult",
        CSASSYMULT.INFO as "Assy Mult",
        CSTESTMULT.INFO as "Test Mult",
        CSPACKMULT.INFO as "Pack Mult",
        CSMACH1.INFO as "Machine 1",
        CSMACH1SETUP.INFO as "Machine 1 Setup",
        CSMACH1MULT.INFO as "Machine 1 Mult",
        CSMACH2.INFO as "Machine 2",
        CSMACH2SETUP.INFO as "Machine 2 Setup",
        CSMACH2MULT.INFO as "Machine 2 Mult"

FROM PART
        left join BOM on PART.DEFAULTBOMID = BOM.ID
        left join (SELECT *
                    FROM CUSTOMSET
                    WHERE CUSTOMSET.CUSTOMFIELDID = 64)
            as CSMAKEBUY on PART.ID = CSMAKEBUY.RECORDID
        left join (SELECT *
                    FROM CUSTOMSET
                    WHERE CUSTOMSET.CUSTOMFIELDID = 111)
            as CSPRODCENTER on BOM.ID = CSPRODCENTER.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 45)
            as CSKITSETUP on BOM.ID = CSKITSETUP.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 46)
            as CSKITMULT on BOM.ID = CSKITMULT.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 102)
            as CSASSYMULT on BOM.ID = CSASSYMULT.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 47)
            as CSTESTMULT on BOM.ID = CSTESTMULT.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 48)
            as CSPACKMULT on BOM.ID = CSPACKMULT.RECORDID
        left join (SELECT *
                    FROM CUSTOMSET
                    WHERE CUSTOMSET.CUSTOMFIELDID = 104)
            as CSMACH1 on BOM.ID = CSMACH1.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 105)
            as CSMACH1SETUP on BOM.ID = CSMACH1SETUP.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 108)
            as CSMACH1MULT on BOM.ID = CSMACH1MULT.RECORDID
        left join (SELECT *
                    FROM CUSTOMSET
                    WHERE CUSTOMSET.CUSTOMFIELDID = 106)
            as CSMACH2 on BOM.ID = CSMACH2.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 107)
            as CSMACH2SETUP on BOM.ID = CSMACH2SETUP.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 109)
            as CSMACH2MULT on BOM.ID = CSMACH2MULT.RECORDID


WHERE CSMAKEBUY.INFO = "Make"
        and BOM.ACTIVEFLAG = true
                    
"""

In [ ]:
bomList = pd.read_clipboard()

In [ ]:
# trying to figure out where the utf errors are.  here we go ...

In [ ]:
import sys

In [ ]:
sys.path.insert(0, 'Z:\Python projects\FishbowlAPITestProject')
import connecttest

In [ ]:
for each in range(380, len(bomList['BOM'])):
    #print(each)
    bommynum = bomList['BOM'].iat[each]
    print(bommynum)
    query = '''SELECT BOM.NUM as "BOM",
        BOMITEMPARTS.NUM as "ItemPart",
        BOMITEM.DESCRIPTION as "ItemDesc",
        BOM.DESCRIPTION as "Description",
        BOM.NOTE as "Note",
        CUSTOMLISTITEM.NAME as "Type",
        BOMAUTOCREATETYPE.NAME as "AutoCreateType",
        CSPRODCENTER.INFO as "Production Center",
        CSKITSETUP.INFO as "Kit Setup",
        CSKITMULT.INFO as "Kit Mult",
        CSASSYMULT.INFO as "Assy Mult",
        CSTESTMULT.INFO as "Test Mult",
        CSPACKMULT.INFO as "Pack Mult",
        CSMACH1.INFO as "Machine 1",
        CSMACH1SETUP.INFO as "Machine 1 Setup",
        CSMACH1MULT.INFO as "Machine 1 Mult",
        CSMACH2.INFO as "Machine 2",
        CSMACH2SETUP.INFO as "Machine 2 Setup",
        CSMACH2MULT.INFO as "Machine 2 Mult"

-- FROM PART
--        left join BOM on PART.DEFAULTBOMID = BOM.ID
FROM BOM
        left join (SELECT *
                    FROM CUSTOMSET
                    WHERE CUSTOMSET.CUSTOMFIELDID = 111)
            as CSPRODCENTER on BOM.ID = CSPRODCENTER.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 45)
            as CSKITSETUP on BOM.ID = CSKITSETUP.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 46)
            as CSKITMULT on BOM.ID = CSKITMULT.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 102)
            as CSASSYMULT on BOM.ID = CSASSYMULT.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 47)
            as CSTESTMULT on BOM.ID = CSTESTMULT.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 48)
            as CSPACKMULT on BOM.ID = CSPACKMULT.RECORDID
        left join (SELECT *
                    FROM CUSTOMSET
                    WHERE CUSTOMSET.CUSTOMFIELDID = 104)
            as CSMACH1 on BOM.ID = CSMACH1.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 105)
            as CSMACH1SETUP on BOM.ID = CSMACH1SETUP.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 108)
            as CSMACH1MULT on BOM.ID = CSMACH1MULT.RECORDID
        left join (SELECT *
                    FROM CUSTOMSET
                    WHERE CUSTOMSET.CUSTOMFIELDID = 106)
            as CSMACH2 on BOM.ID = CSMACH2.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 107)
            as CSMACH2SETUP on BOM.ID = CSMACH2SETUP.RECORDID
        left join (SELECT *
                    FROM CUSTOMDECIMAL
                    WHERE CUSTOMDECIMAL.CUSTOMFIELDID = 109)
            as CSMACH2MULT on BOM.ID = CSMACH2MULT.RECORDID
        left join CUSTOMLISTITEM on BOM.TYPEID = CUSTOMLISTITEM.ID
        left join BOMAUTOCREATETYPE on BOM.AUTOCREATETYPEID = BOMAUTOCREATETYPE.ID
        left join BOMITEM on BOM.ID = BOMITEM.BOMID
        left join (SELECT PART.ID,
                          PART.NUM
                    FROM PART)
            as BOMITEMPARTS on BOMITEM.PARTID = BOMITEMPARTS.ID


WHERE -- BOM.ACTIVEFLAG = true
        BOM.NUM = "%s"''' % bommynum
    myresults = connecttest.create_connection_second_option(query)
    myexcel = connecttest.makeexcelsheet(myresults)
    connecttest.save_workbook(myexcel, csvDir, 'holdExcel.xlsx')
    excel = pd.read_excel(os.path.join(csvDir, 'holdExcel.xlsx'))
    if len(excel) == 0:
        print('''
        OOOOOOOOOOOOO
        OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
        OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
        ooooooooooooooooooooooooooooooooooooooooooooooo
        OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
        OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO''')
    excel.to_csv(os.path.join(csvDir, 'holdCSV.csv'), index=False)
    csv = pd.read_csv(os.path.join(csvDir, 'holdCSV.csv'))